In [1]:
!pip3 install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 981 kB 44.9 MB/s 
     |████████████████████████████████| 263 kB 50.2 MB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
     |████████████████████████████████| 1.3 MB 50.0 MB/s 
     |████████████████████████████████| 468 kB 48.7 MB/s 
     |████████████████████████████████| 880 kB 62.2 MB/s 
     |████████████████████████████████| 120 kB 58.6 MB/s 
     |████████████████████████████████| 6.6 MB 41.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.7-py3-none-any.whl size=25312842 sha256=37833f90920ac61f34b2659e1cb181bfc3789bbe56e16e1997b2c796f49d4ada
  Stored in directory: /root/.cache/pip/wheels/e5/98/8e/ce355dcb92451e85fab93f7ea2da068843e93e703928cd06fb
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=e1848825ee93cc9223974a6f

In [2]:
import missingno as msno
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
import spacy
import tensorflow as tf
from wordcloud import WordCloud, STOPWORDS 
import ktrain
from ktrain import text

from collections import Counter
%matplotlib inline

In [3]:
from transformers import TFDistilBertModel

bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [4]:
import tensorflow.keras.layers as L
seq_len = 350

In [5]:
inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32)

In [6]:
attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32)

In [7]:
attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32)

In [8]:
last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0]

In [9]:
out = last_hidden_state[:,0,:]

In [10]:
bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

In [11]:
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re # regular expression libary.
import nltk # Natural Language toolkit
from nltk import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

In [13]:
import csv
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv ("/content/drive/MyDrive/tripadvisor_hotel_reviews.csv")
print (df)

Mounted at /content/drive
                                                  Review  Rating
0      nice hotel expensive parking got good deal sta...       4
1      ok nothing special charge diamond member hilto...       2
2      nice rooms not 4* experience hotel monaco seat...       3
3      unique, great stay, wonderful time hotel monac...       5
4      great stay great stay, went seahawk game aweso...       5
...                                                  ...     ...
20486  best kept secret 3rd time staying charm, not 5...       5
20487  great location price view hotel great quick pl...       4
20488  ok just looks nice modern outside, desk staff ...       2
20489  hotel theft ruined vacation hotel opened sept ...       1
20490  people talking, ca n't believe excellent ratin...       2

[20491 rows x 2 columns]


In [14]:
X = df['Review'].copy()
y = df['Rating'].copy()

In [15]:
encoding = {1: 0,
            2: 1,
            3: 2,
            4: 3,
            5: 4
           }

labels = ['1', '2', '3', '4', '5']

y = df['Rating'].copy()
y.replace(encoding, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X.values.tolist(), y, test_size=.33, random_state=67, stratify=y)

print("Examples in train data: {}".format(len(train_texts)))
print("Examples in test data: {}".format(len(test_texts)))

Examples in train data: 13728
Examples in test data: 6763


In [17]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

seq_len = 350

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=seq_len)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=seq_len)

input_ids_train = np.array(train_encodings['input_ids']) 
attention_mask_train = np.array(train_encodings['attention_mask'])

input_ids_test = np.array(test_encodings['input_ids']) 
attention_mask_test = np.array(test_encodings['attention_mask'])

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [18]:
exp_sen = 1

print("\nExample:\n")
print("Sentence:\n{}".format(train_texts[exp_sen]))
print("\nAfter tokenizing :\n{}".format(tokenizer.encode(train_texts[exp_sen])))
print("\nAfter padding :\n{}".format(input_ids_train[exp_sen]))


Example:

Sentence:
brilliant berlin brilliant hotel just berlin hotel great, helpful friendly staff clean equipped rooms little bedtime bed really located access touristy stuff hire bike hire companies 12 euros day travel afield loads cycle paths way travel berlin, great u-bahn s-bahn bus access city hotel, little mexican restaurant hotel great tasty filling meal cafe stressemann strasse stressemann cafe does great cheap, breakfasts, hotel restaurant bit expensive food really good service fantastic moevenpick ice cream die, fab trip,  

After tokenizing :
[101, 8235, 4068, 8235, 3309, 2074, 4068, 3309, 2307, 1010, 14044, 5379, 3095, 4550, 6055, 4734, 2210, 2793, 7292, 2793, 2428, 2284, 3229, 7538, 2100, 4933, 10887, 7997, 10887, 3316, 2260, 19329, 2154, 3604, 28697, 14273, 15665, 5402, 10425, 2126, 3604, 4068, 1010, 2307, 1057, 1011, 17392, 1055, 1011, 17392, 3902, 3229, 2103, 3309, 1010, 2210, 4916, 4825, 3309, 2307, 11937, 21756, 8110, 7954, 7668, 6911, 17545, 2358, 8180, 3366, 691

In [19]:
# Creating bert model
inp_ids = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32) # Shape:(batch_size, seq_len)
last_hidden_state = bert(inp_ids,attention_mask=attention_mask)[0] # Shape:(batch_size, seq_len, 768)
out = last_hidden_state[:,0,:] # Shape:(Batch_size, 768)

bert_model = tf.keras.Model(inputs=[inp_ids, attention_mask], outputs=out)

# Passing data from bert pretrained model and extracting the final state.

print("Passing train data")
bert_output_train = bert_model.predict(
    [input_ids_train,attention_mask_train], batch_size=16, verbose=1)

print("Passing test data")
bert_output_test = bert_model.predict(
    [input_ids_test,attention_mask_test], batch_size=16, verbose=1)

Passing train data
858/858 [==============================] - 177s 205ms/step
Passing test data
423/423 [==============================] - 89s 208ms/step


In [21]:
print("Bert output train: {}".format(bert_output_train.shape))
print("Bert output test: {}".format(bert_output_test.shape))

Bert output train: (13728, 768)
Bert output test: (6763, 768)


In [ ]:
bert_output_test.get_barchart_top5words_by_topic()